# Simulating Many Sources in `castor_etc`

Isaac Cheng - April 2022

This notebook shows how to process many point sources using the `castor_etc` Python
package. Note that we do not yet support generating multiple sources in the same aperture
(but you can always simulate your own and [use your own data](./custom_source.ipynb) with
the ETC!).


In [1]:
import astropy.units as u
import numpy as np
import pandas as pd
from IPython.display import display

from castor_etc.background import Background
from castor_etc.photometry import Photometry
from castor_etc.sources import PointSource
from castor_etc.telescope import Telescope

We will use the same `Telescope` and (sky) `Background` objects for all of our photometry
calculations.


In [2]:
MyTelescope = Telescope()
MyBackground = Background()
MyBackground.add_geocoronal_emission("avg")

Now we will generate many different point sources at different AB magnitudes, then find
the time required (in seconds) to achieve a signal-to-noise ratio of 5 and 10 in each
passband.


In [3]:
TARGET_SNRS = [5, 10]
AB_MAGS = np.arange(22, 29, 0.5)
REDDENING = 0.0

wavelengths = np.arange(90, 1200) * u.nm  # generate the spectrum over these wavelengths

# Make some dictionaries to store the results of our calculations
times_to_one_snr = dict.fromkeys(AB_MAGS)
times_to_reach_snrs = {snr: times_to_one_snr.copy() for snr in TARGET_SNRS}

for ab_mag in AB_MAGS:
    #
    # Initialize a point source and generate a flat spectrum in units of AB magnitude
    #
    MySource = PointSource()
    MySource.generate_uniform(wavelengths=wavelengths, value=ab_mag, unit="ABmag")
    #
    # Initialize a Photometry object and use an optimal aperture for a point source
    #
    MyPhot = Photometry(MyTelescope, MySource, MyBackground)
    MyPhot.use_optimal_aperture(quiet=True)  # don't print encircled energy fraction
    for snr in TARGET_SNRS:
        #
        # Find the time, in seconds, required to reach the specified signal-to-noise ratio
        #
        times_to_reach_snrs[snr][ab_mag] = MyPhot.calc_snr_or_t(
            snr=snr, reddening=REDDENING, quiet=True
        )

In [4]:
for snr in TARGET_SNRS:
    print(f"Times (s) required to reach S/N = {snr}")
    display(pd.DataFrame(times_to_reach_snrs[snr]).round(2).T)

Times (s) required to reach S/N = 5


,uv,u,g
22.0,4.02,4.16,3.45
22.5,6.37,6.61,5.51
23.0,10.11,10.50,8.84
23.5,16.04,16.71,14.28
24.0,25.49,26.66,23.31
24.5,40.55,42.69,38.68
25.0,64.65,68.78,65.75
25.5,103.44,111.85,115.71
26.0,166.40,184.50,213.34
26.5,269.94,310.92,416.34


Times (s) required to reach S/N = 10


,uv,u,g
22.0,12.70,13.17,10.97
22.5,20.15,20.92,17.59
23.0,31.98,33.29,28.37
23.5,50.80,53.08,46.19
24.0,80.79,84.95,76.34
24.5,128.76,136.69,128.91
25.0,205.86,221.82,224.47
25.5,330.78,364.65,407.16
26.0,535.62,611.09,776.52
26.5,877.61,1052.76,1565.28
